<a href="https://colab.research.google.com/github/MT121201/Computer_Vision/blob/main/Face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns

from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from glob import glob

import xml.etree.ElementTree as ET

In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d andrewmvd/face-mask-detection

100% 397M/398M [00:19<00:00, 23.9MB/s]
100% 398M/398M [00:19<00:00, 21.0MB/s]


In [5]:
! unzip -q face-mask-detection

In [6]:
images = glob(os.path.join("/content/images/*.png"))

In [7]:
anns = glob(os.path.join("/content/annotations/*.xml"))

In [8]:
ann_data = pd.DataFrame()
for ann in anns:
    tree = ET.parse(ann)
    root = tree.getroot()
    counter = 0
    dic = dict()
    dic["Image name"] =  root[1].text
    for i in range(4, len(root)):
        tmp = []
        tmp.append(root[i][0].text)##name
        for point in root[i][5]:
            tmp.append(point.text)##bnd box
        dic["wear_mask" + str(counter)] = tmp
        counter += 1
    # ann_data = pd.concat(dic, ignore_index= True)
    ann_data = ann_data.append(dic, ignore_index= True)
    counter = 0

<ipython-input-8-80d28e0d2b12>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ann_data = ann_data.append(dic, ignore_index= True)
<ipython-input-8-80d28e0d2b12>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ann_data = ann_data.append(dic, ignore_index= True)
<ipython-input-8-80d28e0d2b12>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ann_data = ann_data.append(dic, ignore_index= True)
<ipython-input-8-80d28e0d2b12>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ann_data = ann_data.append(dic, ignore_index= True)
<ipython-input-8-80d28e0d2b12>:16: FutureWarning: The frame.append method is deprecated and will be removed from

In [9]:
ann_data =ann_data.fillna(5)

In [10]:
ann_data.head()

,Image name,wear_mask0,wear_mask1,wear_mask2,wear_mask3,wear_mask4,wear_mask5,wear_mask6,wear_mask7,wear_mask8,...,wear_mask105,wear_mask106,wear_mask107,wear_mask108,wear_mask109,wear_mask110,wear_mask111,wear_mask112,wear_mask113,wear_mask114
0,maksssksksss716.png,"[without_mask, 101, 174, 209, 299]",5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1,maksssksksss323.png,"[with_mask, 322, 257, 368, 346]",5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
2,maksssksksss125.png,"[without_mask, 118, 151, 216, 279]",5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
3,maksssksksss394.png,"[with_mask, 142, 13, 249, 133]",5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,maksssksksss651.png,"[with_mask, 186, 41, 217, 77]",5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [11]:
image_path = "/content/images/"
labels = []
data = []
classes = ["without_mask", "mask_weared_incorrect", "with_mask"]

In [12]:
from keras.utils import img_to_array
for index, row in ann_data.iterrows():
    img = cv2.imread(os.path.join(image_path, row[0]))
    cv2.resize(img, (255,255))
    for obj in row[1:]:
        if obj != 5:##fill null with 5 above
            label = obj[0]
            obj[0] = obj[0].replace(str(label), str(classes.index(label)))
            obj = [int(each) for each in obj]
            face = img[obj[2]:obj[4], obj[1]: obj[3]]
            face = cv2.resize(face, (224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
            data.append(face)
            labels.append(label)
            # if(label == "mask_weared_incorrect"):
            #     data.append(face)
            #     labels.append(labels)

In [13]:
data = np.array(data , dtype="float32")
labels = np.array(labels)

In [14]:
print(data.shape)
print(labels.shape)

(4072, 224, 224, 3)
(4072,)


In [15]:
type(labels)

numpy.ndarray

In [16]:
from keras.utils.np_utils import to_categorical
lable_encode = LabelEncoder()
labels = lable_encode.fit_transform(labels)
labels = to_categorical(labels)

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=.2, stratify =labels, random_state = 42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=.2, random_state = 42)

In [27]:
model = Sequential()

model.add(Flatten(input_shape = (224 , 224 , 3)))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(50, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(3 , activation = 'softmax'))

In [28]:
INIT_LR = 1e-4
EPOCHS = 15
BS = 1

In [ ]:
opt = optimizers.Adam(lr = INIT_LR , decay = INIT_LR / EPOCHS)
model.compile(loss = "categorical_crossentropy" , optimizer = opt , metrics = ["accuracy"])
H = model.fit(X_train, Y_train , steps_per_epoch = len(X_train) // BS , validation_data = (X_val , Y_val) , validation_steps = len(X_val) // BS , epochs = EPOCHS)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
import numpy as np
import seaborn as sns

In [ ]:
y_pred = model.predict(X_test)
y_pred_new = np.argmax(y_pred , axis = 1) # To get the index (The class numer) of the predicted class
y_test_new = np.argmax(Y_test , axis = 1)

In [ ]:
Confusion_Mtrx = metrics.confusion_matrix(y_test_new , y_pred_new)
#true_positive , false_positive, false_negative, true_negative = Confusion_Mtrx.ravel()

plt.figure(figsize=(6,6))
sns.heatmap(Confusion_Mtrx, annot=True, fmt=".0f");
plt.ylabel('Real Value');
plt.xlabel('Predicted Values');
plt.title('Accuracy {0:.2f}'.format(metrics.accuracy_score(y_test_new, y_pred_new)));